In [1]:
import os, sys
sys.path.append('../src/')

from copulae1 import *

import json
with open('../src/configs/config_CRIX_future_Open_v3_MM.json','r') as file:
    config = json.load(file)
config
data_name = config['data_name']
calibration_method  = config['calibration_method']
OHR = pd.read_hdf('../results/'+data_name +'/'+calibration_method+'/best_h.h5')
if not os.path.exists("../results/" + data_name + "/MLE/reports"):
	print("Create new folder for results")
	os.mkdir("../results/" + data_name + "/MLE/reports")
	os.mkdir("../results/" + data_name + "/MM/reports")
    
if calibration_method == "MLE":
    report_path = "../results/" + data_name + "/MLE/reports/"
    result_path = "../results/" + data_name + "/MLE/"
elif calibration_method == "MM":
    report_path = "../results/" + data_name + "/MM/reports/"
    result_path = "../results/" + data_name + "/MM/"

Create new folder for results


In [2]:
OHR

0.csv                                                      \
                 Variance  ERM k=10 ES q=0.01 ES q=0.05 VaR q=0.01 VaR q=0.05   
Gaussian         0.971094  0.922852  1.202832  0.958789   0.937207   0.865332   
t_Copula         0.946582  0.911816  1.228223  0.940820   0.843652   0.877441   
t_Copula_Capped  0.965723  0.919238  1.230762  0.957715   0.904687   0.870215   
Clayton          1.024121  0.923535  1.195703  0.887109   0.803516   0.958008   
Frank            0.878516  0.878809  0.994629  0.877246   0.887891   0.865527   
Gumbel           0.944824  0.980664  1.247852  1.035449   1.108984   0.943750   
Plackett         0.914648  0.904883  1.132129  0.912305   0.883887   0.878711   
Gauss Mix Indep  0.913477  0.913086  1.156055  0.933691   0.612305   0.928516   

                    1.csv                                ...    95.csv  \
                 Variance  ERM k=10 ES q=0.01 ES q=0.05  ... ES q=0.01   
Gaussian         0.977051  0.918457  1.207520  0.948340  ...  1.364844   
t_Copula         0.955176  0.910156  1.231152  0.934277  ...  1.303809   
t_Copula_Capped  0.972852  0.916406  1.235352  0.948242  ...  1.303809   
Clayton          1.021777  0.916895  1.197949  0.887988  ...  1.089746   
Frank            0.913574  0.921094  1.150488  0.936816  ...  1.407520   
Gumbel           0.956543  0.979687  1.243848  1.030762  ...  1.659082   
Plackett         0.922363  0.904590  1.123828  0.909277  ...  1.313965   
Gauss Mix Indep  0.910352  0.913574  1.144141  0.915430  ...  1.039453   

                                                   96.csv                      \
                ES q=0.05 VaR q=0.01 VaR q=0.05  Variance  ERM k=10 ES q=0.01   
Gaussian         1.201367   1.225195   1.021973  0.951562  1.106348  1.327148   
t_Copula         1.179004   1.177539   1.024805  0.917187  1.064746  1.160156   
t_Copula_Capped  1.179004   1.177539   1.024805  0.917187  1.064746  1.160156   
Clayton          0.911328   0.913477   0.895508  0.952441  0.899316  0.897656   
Frank            1.161523   1.183398   1.017383  0.979004  1.111426  1.370508   
Gumbel           1.367188   1.472852   1.198438  0.904395  1.200977  1.604785   
Plackett         1.112305   1.107422   1.021777  0.923047  1.063086  1.249512   
Gauss Mix Indep  1.123047   1.037793   1.089453  0.862012  1.023047  0.919434   

                                                 
                ES q=0.05 VaR q=0.01 VaR q=0.05  
Gaussian         1.180957   1.212500   1.028223  
t_Copula         1.118262   1.131445   1.029688  
t_Copula_Capped  1.118262   1.131445   1.029688  
Clayton          0.868262   0.842383   0.898340  
Frank            1.163281   1.216016   1.031250  
Gumbel           1.346777   1.460938   1.168750  
Plackett         1.099512   1.143555   1.018848  
Gauss Mix Indep  1.001660   0.975977   1.103906  

[8 rows x 582 columns]

In [370]:
natp_path = '../_mathematica/data/'
natp_ls = os.listdir(natp_path)
natp_h = [l for l in natp_ls if l.endswith('_h.csv')]
file_names = [l.replace('_h','') for l in natp_h]
OHR_combined = []
for i in range(len(natp_h)):
    _natp_h = pd.DataFrame(open(natp_path+natp_h[i], 'r').readlines())
    _natp_h = _natp_h.iloc[1:7,:]
    file_name = natp_h[i].replace('_h','')
    for i in range(len(_natp_h)):
        _natp_h.iloc[i,0] = np.float32(_natp_h.iloc[i,0].replace('\n',''))
    _natp_h.columns = ['NIG_factor']
    _natp_h.index = ['Variance', 'VaR q=0.01', 'VaR q=0.05', 'ES q=0.01', 'ES q=0.05', 'ERM k=10']
    _natp_h = _natp_h.T
    
    OHR_combined.append(OHR.loc[:,file_name].append(_natp_h))
OHR = pd.concat(OHR_combined, axis=1, keys=file_names).dropna(axis=1)

/Users/francis/opt/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:6692: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)


In [3]:
# Copula_names = ['Gaussian', 't_Copula', 'Clayton', 'Frank', 'Gumbel', 'Plackett', 'Gauss Mix Indep', 'NIG_factor']
Copula_names = ['Gaussian', 't_Copula', 'Clayton', 'Frank', 'Gumbel', 'Plackett', 'Gauss Mix Indep']

OHR = OHR.loc[Copula_names,:]

In [4]:
temp = []
for C in Copula_names:
    hs_by_Copula = OHR.loc[C:C,:].melt()
    hs_by_Copula.columns = ['file_name', 'risk_measure', 'OHR']
    hs_by_Copula.loc[:, 'copula'] = C
    temp.append(hs_by_Copula)

OHR_to_math = pd.concat(temp, axis=0)

In [5]:
v5_path = '../processed_data/'+config['data_name'].replace('3', '5')+'/test/'
v3_path = '../processed_data/'+config['data_name']+'/test/'

v5_files = os.listdir(v5_path)
v3_files = os.listdir(v3_path)

In [6]:
pd.read_csv(v5_path+'0.csv').tail()

,Unnamed: 0,Date,CRIX,future,log return CRIX,log return future
95,95,2020-09-18 00:00:00+00:00,31525.169649,10975.0,0.002718,-0.006358
96,96,2020-09-17 00:00:00+00:00,31439.598392,11045.0,0.014015,0.014133
97,97,2020-09-16 00:00:00+00:00,31002.050083,10890.0,0.005421,0.012939
98,98,2020-09-15 00:00:00+00:00,30834.443871,10750.0,0.026179,0.041307
99,99,2020-09-14 00:00:00+00:00,30037.716520,10315.0,0.001040,-0.005318


In [7]:
pd.read_csv(v3_path+'19.csv') 

,Unnamed: 0,Date,CRIX,future,log return CRIX,log return future
0,95,2020-09-18 00:00:00+00:00,31525.169649,10975.0,0.002718,-0.006358
1,96,2020-09-17 00:00:00+00:00,31439.598392,11045.0,0.014015,0.014133
2,97,2020-09-16 00:00:00+00:00,31002.050083,10890.0,0.005421,0.012939
3,98,2020-09-15 00:00:00+00:00,30834.443871,10750.0,0.026179,0.041307
4,99,2020-09-14 00:00:00+00:00,30037.716520,10315.0,0.001040,-0.005318


In [8]:
OHR_to_use = OHR_to_math.copy()
OHR_to_use.file_name = OHR_to_use.file_name.apply(lambda x: int(x.replace('.csv',''))-19) # shift file names
OHR_to_use = OHR_to_use.loc[OHR_to_use.file_name >0,:] # remove redudant rows
OHR_to_use.file_name = OHR_to_use.file_name.apply(lambda x: str(x)+'.csv')

In [9]:
risk_measure = list(OHR_to_use.risk_measure.unique())
ls = list(OHR_to_use.file_name.unique())

In [10]:
OHR_to_use.loc[:,"HE"] = 0 # Add a new column for results of HE

In [11]:
OHR_to_use.head()

,file_name,risk_measure,OHR,copula,HE
78,1.csv,Variance,0.895117,Gaussian,0
79,1.csv,ERM k=10,0.875879,Gaussian,0
80,1.csv,ES q=0.01,0.989648,Gaussian,0
81,1.csv,ES q=0.05,0.898340,Gaussian,0
82,1.csv,VaR q=0.01,0.924609,Gaussian,0


In [12]:
def hedging_effectiveness2(rm, rs, rf, h):
    rh = rs - h*rf
    if rm == 'ERM k=10':
        return(1-ERM_estimate_trapezoidal(10, rh)/ERM_estimate_trapezoidal(10, rs))

    elif rm == 'ES q=0.01':
        return(1-ES(0.01, rh)/ES(0.01, rs))

    elif rm == 'ES q=0.05':
        return(1-ES(0.05, rh)/ES(0.05, rs))

    elif rm == 'VaR q=0.01':
        return(1-VaR(0.01, rh)/VaR(0.01, rs))

    elif rm == 'VaR q=0.05':
        return(1-VaR(0.05, rh)/VaR(0.05, rs))

    elif rm == 'Variance':
        return(1-Variance(rh)/Variance(rs))

In [13]:
def rename_rm(rm):
    rm = rm.replace('q=','')
    rm = rm.replace('0.01', '99%')
    rm = rm.replace('0.05', '95%')
    return rm

In [14]:
config['spot_name']

'log return CRIX'

In [15]:
for i in range(len(OHR_to_use)):
    try:
        file = OHR_to_use.iloc[i,:].file_name
        rs = pd.read_csv(v5_path+file).loc[:,config['spot_name']]
        rf = pd.read_csv(v5_path+file).loc[:,config['future_name']]
        h = OHR_to_use.iloc[i, :].OHR
        rm = OHR_to_use.iloc[i,:].risk_measure
        OHR_to_use.iloc[i,-1] = hedging_effectiveness2(rm, rs, rf, h)
    except:
        OHR_to_use.iloc[i,-1] = np.NAN
        print(file +' is not in ls')
        continue
OHR_to_use.dropna(inplace=True)

In [16]:
i_id = OHR_to_use.risk_measure == rm
j_id = OHR_to_use.copula == C

In [17]:
path = '../latex/_pics/CRIX_future_Open_hedging_effectiveness_100days_with_v3_OHR/'

if os.path.exists(path) == False:
    os.mkdir(path)
    
for rm in risk_measure:
    i_id = OHR_to_use.risk_measure == rm
    
    plt.figure(figsize=(10,5))
    plt.title('Out-of-Sample Hedging Effectiveness of %s' %rename_rm(rm))
    
    for j, C in enumerate(Copula_names):
        j_id = OHR_to_use.copula == C
        plt.boxplot(OHR_to_use.loc[i_id&j_id,'HE'], positions=[j])
    plt.ylim((0,1))
    plt.xticks(range(len(Copula_names)), Copula_names)
    plt.savefig(path+'%s_100days_with_v3_OHR.pdf'%(rename_rm(rm).replace(' ', '_')))
    plt.close()

In [18]:
import itertools
a = ['HR 1']
b = risk_measure.copy()
c = ls.copy()
d = [1]
naive = pd.DataFrame(list(itertools.product(*[a,b,c,d])))
naive.columns = ['copula', 'risk_measure', 'file_name', 'OHR']
naive.loc[:,'HE'] = 0

In [19]:
OHR_to_use = OHR_to_use.append(naive)

/Users/francis/opt/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:6692: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)


In [21]:
for i in range(len(OHR_to_use)):
    try:
        file = OHR_to_use.iloc[i,:].file_name
        rs = pd.read_csv(v5_path+file).loc[:,config['spot_name']]
        rf = pd.read_csv(v5_path+file).loc[:,config['future_name']]
        h = OHR_to_use.iloc[i, :].OHR
        rm = OHR_to_use.iloc[i,:].risk_measure
        OHR_to_use.iloc[i,0] = hedging_effectiveness2(rm, rs, rf, h)
    except:
        OHR_to_use.iloc[i,-1] = np.NAN
        print(file +' is not in ls')
        continue
OHR_to_use.dropna(inplace=True)


In [22]:
path = '../latex/_pics/CRIX_future_hedging_effectiveness_100days_with_v3_OHR_with_naive/'

if os.path.exists(path) == False:
    os.mkdir(path)
    
for rm in risk_measure:
    i_id = OHR_to_use.risk_measure == rm
    
    plt.figure(figsize=(12,5))
    plt.title('Out-of-Sample Hedging Effectiveness of %s' %rename_rm(rm))
    
    for j, C in enumerate(Copula_names):
        j_id = OHR_to_use.copula == C
        plt.boxplot(OHR_to_use.loc[i_id&j_id,'HE'], positions=[j])
   
    j_id = OHR_to_use.copula == 'HR 1'
    plt.boxplot(OHR_to_use.loc[i_id&j_id,'HE'], positions=[j+1])
    plt.ylim((0,1))
    plt.xticks(range(len(Copula_names)+1), Copula_names+['HR 1'])
    plt.savefig(path+'%s_100days_with_v3_OHR_with_naive.pdf'%(rename_rm(rm).replace(' ', '_')))
    plt.close()